In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [20]:
train_set=np.load("dataset/train.npz")
val_set=np.load("dataset/val.npz")

## Construct Baseline

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [26]:
def to_csv(data_set,filepath):
    x = data_set['x'] #feature matrix
    y = data_set['y'] #label matrix
    locations = data_set['locations'] #location matrix
    times = data_set['times'] #time matrix
    print(f'the shape of x: {x.shape}\nthe shape of y: {y.shape}\nthe shape of locations: {locations.shape}\nthe shape of times: {times.shape}')

    feature=np.reshape(x,(x.shape[0],-1))
    times = times.reshape(times.shape[0],1)
    train_data = np.concatenate((feature, locations, times),axis=1)
    print(f'the concatenate data shape: {train_data.shape}')
    df = pd.DataFrame(train_data)
    df['label']=y
    df.to_csv(filepath,index=False) #save the data to csv


In [29]:
def smape(A, F): # A: Actual, F: Forecast
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
to_csv(train_set, "dataset/train.csv")

In [27]:
to_csv(val_set, "dataset/val.csv")

the shape of x: (18000, 8, 49)
the shape of y: (18000, 1)
the shape of locations: (18000, 2)
the shape of times: (18000,)
the concatenate data shape: (18000, 395)


In [10]:
train_data = TabularDataset('dataset/train.csv')
val_data = TabularDataset('dataset/val.csv')

Loaded data from: dataset/train.csv | Columns = 396 / 396 | Rows = 72000 -> 72000
Loaded data from: dataset/val.csv | Columns = 396 / 396 | Rows = 72000 -> 72000


In [ ]:
# train
predictor = TabularPredictor(label='label', 
                             eval_metric='root_mean_squared_error')\
            .fit(train_data,num_stack_levels=3,num_bag_folds=3)

In [ ]:
# pred
predictions = predictor.predict(val_data.drop(columns=['label']))
y_true=val_data["label"]

In [19]:
mean_squared_error(y_true, predictions)

239.17158333333333

In [31]:
smape(y_true, predictions)

145.4138888888889